In [2]:
TRAIN_INPUT_LOC = 'dataset/train_motion_data'
TEST_INPUT_LOC = 'dataset/test_motion_data'
TEMP_DATA = './dataset/temp/temp.bvh'
OUTPUT_LOC = 'dataset/acceleration_data'
FRAME_RATE = 30

In [3]:
import joblib
import smplx
import os
import numpy as np
import scipy.signal as signal
import pandas as pd
import sys

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
label_map = {
    'seated_march': 0,
    'seated_shoulder_abduction': 1,
    'sit_to_stand': 2
}


In [5]:
import smpl2bvh as s2b
from imusim.platforms.imus import IdealIMU, Orient3IMU
from imusim.simulation.base import Simulation
from imusim.behaviours.imu import BasicIMUBehaviour
from imusim.io.bvh import loadBVHFile
from imusim.trajectories.rigid_body import SplinedBodyModel
from imusim.environment.base import Environment
from imusim.simulation.calibrators import ScaleAndOffsetCalibrator

In [7]:
s2b.smpl2bvh('./body_models', './dataset/train_motion_data/sit_to_stand/sit_to_stand-1.pkl', TEMP_DATA, False, fps=FRAME_RATE)


In [40]:
def imu_train_wrist_data(pose_file):
    s2b.smpl2bvh('./body_models', pose_file, TEMP_DATA, False, fps=FRAME_RATE)
    model = loadBVHFile(TEMP_DATA)
    print ('load mocap from ...', TEMP_DATA)
    #print (model)

    # spline intrepolation
    splinedModel = SplinedBodyModel(model)
    startTime = splinedModel.startTime
    endTime = splinedModel.endTime

    # setting sampling period to be same as frame rate can change tho
    frameCount = int(FRAME_RATE * (endTime-startTime))
    samplingPeriod = (endTime - startTime)/ frameCount

    print ('frameCount:', frameCount)
    print ('samplingPeriod:', samplingPeriod)

    if version == 'ideal':
        print ('Simulating ideal IMU.')

        # set simulation
        sim = Simulation()
        sim.time = startTime

        # run simulation
        dict_imu = {}
        imu = IdealIMU()
        imu.simulation = sim
        imu.trajectory = splinedModel.getJoint("Right_wrist")

        BasicIMUBehaviour(imu, samplingPeriod)

        dict_imu["Right_wrist"] = imu

        sim.run(endTime)

    elif version == 'sim':
        print ('Simulating Orient3IMU.')
        
        # set simulation
        env = Environment()
        calibrator = ScaleAndOffsetCalibrator(env, calibSamples, samplingPeriod, calibRotVel)
        sim = Simulation(environment=env)
        sim.time = startTime

        # run simulation
        dict_imu = {}

        imu = Orient3IMU()
        calibration = calibrator.calibrate(imu)
        print ('imu calibration:', "Right_wrist")
        
        imu.simulation = sim
        imu.trajectory = splinedModel.getJoint("Right_wrist")

        BasicIMUBehaviour(imu, samplingPeriod, calibration, initialTime=sim.time)

        dict_imu["Right_wrist"] = imu

        sim.run(endTime)

    # collect sensor values

    imu = dict_imu["Right_wrist"]

    if version == 'ideal':    
        acc_seq = imu.accelerometer.rawMeasurements.values.T
        gyro_seq = imu.gyroscope.rawMeasurements.values.T
    elif version == 'sim':
        acc_seq = imu.accelerometer.calibratedMeasurements.values.T
        gyro_seq = imu.gyroscope.calibratedMeasurements.values.T
        
    imu_data = np.concatenate((acc_seq, gyro_seq), axis=1)
    return imu_data
        

In [41]:
def extract_acceleration_data(rel_input_loc, resample_rate=None, dataset='train'):
    """
    Extracts wrist acceleration data from files within sub-folders of a specified directory, 
    optionally resamples the data, and saves both the acceleration data and labels to NumPy files.

    Args:
        rel_input_loc (str): 
            Relative path to the parent directory containing one or more sub-folders of CSV files. 
            Each sub-folder is associated with a specific label/category.
        resample_rate (float, optional): 
            Desired sample rate (in Hz) to resample each acceleration dataset. 
            If None, no resampling is performed. Defaults to None.
        dataset (str): 
            The type of dataset to process. 
            - If 'train', this function uses the global `wristAcceleration` function, applying a 
              fixed global frame rate (`FRAME_RATE`), rather than reading the sample rate from file. 
            - If 'test', it uses the `testAcceleration` function to extract the actual sample rate 
              from each CSV file before optionally resampling.
    """
    input_loc = os.path.join(os.getcwd(), rel_input_loc)
    output_loc = os.path.join(os.getcwd(), OUTPUT_LOC)

    # Get the list of files in the input directory
    folder_list = os.listdir(input_loc)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_loc):
        os.makedirs(output_loc)

    acceleration_data = []
    label_data = []
    sample_rates = []
    # Process each file
    for idx,folder in enumerate(folder_list):
        folder_loc = os.path.join(input_loc, folder)
        label_num = label_map.get(folder)

        if not os.path.isdir(folder_loc):
            continue

        print(f'Processing folder {idx+1}/{len(folder_list)}: {folder}')

        file_list = os.listdir(folder_loc)
        for file_name in file_list:
            file_loc = os.path.join(folder_loc, file_name)
            if dataset == 'train':
                imu_train_wrist_data(file_loc)



In [42]:
extract_acceleration_data(TRAIN_INPUT_LOC, dataset='train')


Processing folder 1/4: seated_march
load mocap from ... ./dataset/temp/temp.bvh
frameCount: 141
samplingPeriod: 0.033569404255319145
Simulating Orient3IMU.
imu calibration: Right_wrist
Simulating...
Simulated 0.2s of 4.7s (  5%). Estimated time remaining 0.2s
Simulated 0.5s of 4.7s ( 10%). Estimated time remaining 0.2s
Simulated 0.7s of 4.7s ( 15%). Estimated time remaining 0.1s
Simulated 0.9s of 4.7s ( 20%). Estimated time remaining 0.1s
Simulated 1.2s of 4.7s ( 25%). Estimated time remaining 0.1s
Simulated 1.4s of 4.7s ( 30%). Estimated time remaining 0.1s
Simulated 1.7s of 4.7s ( 35%). Estimated time remaining 0.1s
Simulated 1.9s of 4.7s ( 40%). Estimated time remaining 0.1s
Simulated 2.1s of 4.7s ( 45%). Estimated time remaining 0.1s
Simulated 2.4s of 4.7s ( 50%). Estimated time remaining 0.1s
Simulated 2.6s of 4.7s ( 55%). Estimated time remaining 0.1s
Simulated 2.8s of 4.7s ( 60%). Estimated time remaining 0.1s
Simulated 3.1s of 4.7s ( 65%). Estimated time remaining 0.1s
Simulate